# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []

# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}
    """
)

#### Set Keyspace

In [7]:
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [24]:
## Here, we need to create a primary key on sessionid (partition-key) and iteminsession(clustering-key). Records with the same sessionId
## will be in the same partition & will be sorted by itemInSession.

create_table_stmt = """
    CREATE TABLE IF NOT EXISTS events_session(
        sessionId int,
        itemInSession int,
        artist text,
        firstName text,
        lastName text,
        gender text,
        length float,
        level text,
        location text,
        song text,
        userId int,
        PRIMARY KEY ((sessionId), itemInSession)
    )
"""
session.execute(create_table_stmt)


In [25]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
            INSERT INTO events_session(
                sessionId,
                itemInSession,
                artist,
                firstName,
                lastName,
                gender,
                length,
                level,
                location,
                song,
                userId
            )
        """
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[1], line[4], 
                                line[2], float(line[5]), line[6], line[7], line[9], int(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [26]:
query = "SELECT artist, song, length FROM events_session WHERE sessionId=338 AND itemInSession=4"
rows = session.execute(query)
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
## Here, we need to create a primary key on userId & sessionid (partition-keys) and iteminsession(clustering-key).
## Records with the same userId & sessionId will be in the same partition & will be sorted by itemInSession.

create_table_stmt = """
    CREATE TABLE IF NOT EXISTS events_user_session(
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        firstName text,
        lastName text,
        gender text,
        length float,
        level text,
        location text,
        song text,
        PRIMARY KEY ((userId, sessionId), itemInSession)
    )
"""
session.execute(create_table_stmt)


In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
            INSERT INTO events_user_session(
                userId,
                sessionId,
                itemInSession,
                artist,
                firstName,
                lastName,
                gender,
                length,
                level,
                location,
                song
            )
        """
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], 
                                line[2], float(line[5]), line[6], line[7], line[9]))

In [13]:
query = "SELECT artist, song, firstName, lastName FROM events_user_session WHERE userId=10 AND sessionId=182"
rows = session.execute(query)
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [18]:
## Here, we need to create a primary key on song (partition-key) and userId(clustering-key). Records with the same song
## will be in the same partition & will be sorted by userId.

create_table_stmt = """
    CREATE TABLE IF NOT EXISTS events_song(
        song text,
        userId int,
        artist text,
        firstName text,
        lastName text,
        gender text,
        length float,
        level text,
        location text,
        sessionId int,
        itemInSession int,
        PRIMARY KEY ((song), userId)
    )
"""
session.execute(create_table_stmt)

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
            INSERT INTO events_song(
                song,
                userId,
                artist,
                firstName,
                lastName,
                gender,
                length,
                level,
                location,
                sessionId,
                itemInSession
            )
        """
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[0], line[1], line[4], line[2], float(line[5]), line[6], line[7],
                                int(line[8]), int(line[3])))

In [22]:
query = "SELECT firstName, lastName FROM events_song WHERE song='All Hands Against His Own'"
rows = session.execute(query)
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [23]:
tables = ['events_session', 'events_user_session', 'events_song']
for table in tables:
    session.execute(f"drop table {table}")

### Close the session and cluster connection¶

In [37]:
session.shutdown()
cluster.shutdown()